In [ ]:
import duckdb

In [ ]:
conn = duckdb.connect()

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS units (
         name VARCHAR PRIMARY KEY,
         long_name VARCHAR,
         -- symbolic_name VARCHAR,
);
INSERT INTO units VALUES ('cms', 'Cubic Meters Per Second');
INSERT INTO units VALUES ('cfs', 'Cubic Feet Per Second');
INSERT INTO units VALUES ('km2', 'Square Kilometers');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS configurations (
         name VARCHAR PRIMARY KEY,
         type VARCHAR,
         description VARCHAR
);
INSERT INTO configurations VALUES ('nwm30_retro', 'secondary', 'NWM 3.0 Retrospective');
INSERT INTO configurations VALUES ('usgs_observations', 'primary', 'USGS Observations');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS variables (
         name VARCHAR PRIMARY KEY,
         long_name VARCHAR
);
INSERT INTO variables VALUES ('streamflow_hourly_inst', 'Hourly Instantaneous Streamflow');
INSERT INTO variables VALUES ('streamflow_daily_mean', 'Daily Mean Streamflow');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS attributes (
         name VARCHAR PRIMARY KEY,
         unit_name VARCHAR,
         FOREIGN KEY (unit_name) REFERENCES units (name)
);
INSERT INTO attributes VALUES ('drainage_area_km2', 'km2');
INSERT INTO attributes VALUES ('mean_daily_flow_cms', 'cms');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS location_crosswalk (
         primary_location_id VARCHAR PRIMARY KEY,
         secondary_location_id VARCHAR,
);
INSERT INTO location_crosswalk VALUES ('p-1', 's-1');
INSERT INTO location_crosswalk VALUES ('p-2', 's-2');
""")

In [ ]:
conn.sql("""
INSTALL spatial;
LOAD spatial;
CREATE TABLE IF NOT EXISTS locations (
         id VARCHAR PRIMARY KEY,
         name VARCHAR,
         geometry GEOMETRY
);
INSERT INTO locations VALUES ('p-1', 'gage-A', 'POINT(1 1)');
INSERT INTO locations VALUES ('p-2', 'gage-B', 'POINT(2 2)');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS primary_timeseries (
    reference_time DATETIME,
    value_time DATETIME,
    configuration VARCHAR,
    measurement_unit VARCHAR,
    variable_name VARCHAR,
    value FLOAT,
    location_id VARCHAR,
    FOREIGN KEY (configuration) REFERENCES configurations (name),
    FOREIGN KEY (measurement_unit) REFERENCES units (name),
    FOREIGN KEY (variable_name) REFERENCES variables (name),
    FOREIGN KEY (location_id) REFERENCES location_crosswalk (primary_location_id)
);
""")

In [ ]:
conn.sql("""
INSERT INTO primary_timeseries VALUES ('2024-01-01 12:00:00', '2024-01-01 12:00:00', 'usgs_observations', 'cms', 'streamflow_daily_mean', 0.12, 'p-1');
INSERT INTO primary_timeseries VALUES ('2024-01-02 12:00:00', '2024-01-01 12:00:00', 'usgs_observations', 'cms', 'streamflow_daily_mean', 2.1, 'p-1');
""")


In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS secondary_timeseries (
    reference_time DATETIME,
    value_time DATETIME,
    configuration VARCHAR,
    measurement_unit VARCHAR,
    variable_name VARCHAR,
    value FLOAT,
    location_id VARCHAR,
    FOREIGN KEY (configuration) REFERENCES configurations (name),
    FOREIGN KEY (measurement_unit) REFERENCES units (name),
    FOREIGN KEY (variable_name) REFERENCES variables (name),
    FOREIGN KEY (location_id) REFERENCES location_crosswalk (secondary_location_id)
);
""")

In [ ]:
conn.sql("""
INSERT INTO secondary_timeseries VALUES ('2024-01-01 12:00:00', '2024-01-01 12:00:00', 'nwm30_retro', 'cms', 'streamflow_daily_mean', 0.2, 's-1');
INSERT INTO secondary_timeseries VALUES ('2024-01-02 12:00:00', '2024-01-01 12:00:00', 'nwm30_retro', 'cms', 'streamflow_daily_mean', 2.3, 's-1');
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS location_attributes(
         attribute_name VARCHAR PRIMARY KEY,
         value VARCHAR,
         location_id VARCHAR,
         FOREIGN KEY (attribute_name) REFERENCES attributes (name),
         FOREIGN KEY (location_id) REFERENCES locations (id)
);
""")

In [ ]:
conn.sql("""
CREATE TABLE IF NOT EXISTS joined_timeseries(
    reference_time DATETIME,
    value_time DATETIME,
    configuration VARCHAR,
    measurement_unit VARCHAR,
    variable_name VARCHAR,
    value FLOAT,
    location_id VARCHAR,
    FOREIGN KEY (measurement_unit) REFERENCES units (name)
);
""")

In [ ]:
conn.sql("""
INSERT INTO joined_timeseries VALUES ('2024-01-01 12:00:00', '2024-01-01 12:00:00', 'config', 'cms', 'var', 0.12, '12345');
""")

In [ ]:
conn.sql("""
SELECT * FROM  joined_timeseries;
""")